# Exercice 1

En utilisant les données US airline créer un notebook(Je vous recommande d’utiliser google
colab et installer package jupyter-dash) que vous allez nommé Prenom_Nom_exercice1:
Créer un dashboard pour analyser les retards des vols
Questions:
1. créer un champ de saisie pour choisir l'année ( Alternative créer un dropdown pour
choisir l'année)
2. Moyenne mensuelle des retards des transporteurs par compagnie aérienne
déclarante pour l'année donnée.
3. Retard météorologique moyen mensuel par compagnie aérienne déclarante pour
l'année donnée.
4. Moyenne mensuelle des retards du système

<hr>

## Ecercice réalisé par :

**Papa Ba GAYE**

In [9]:
# !pip install dash
# !pip install jupyter-dash
# !pip install dash-bootstrap-components

In [2]:
import pandas as pd
import dash
from jupyter_dash import JupyterDash
from dash import dash_table
import pandas as pd
from dash import  dcc
import dash_bootstrap_components as dbc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
path_data = "/content/drive/MyDrive/Mes Codes/Visualisation de Données/Projet Data Viz/Individuel/airline.csv"
data = pd.read_csv(path_data)

In [5]:
data.head()

,Unnamed: 0,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,...,Div1Airport,Div1AirportID,Div1AirportSeqID,Div1WheelsOn,Div1TotalGTime,Div1LongestGTime,Div1WheelsOff,Div1TailNum,Div2Airport,Div2AirportID,Div2AirportSeqID,Div2WheelsOn,Div2TotalGTime,Div2LongestGTime,Div2WheelsOff,Div2TailNum,Div3Airport,Div3AirportID,Div3AirportSeqID,Div3WheelsOn,Div3TotalGTime,Div3LongestGTime,Div3WheelsOff,Div3TailNum,Div4Airport,Div4AirportID,Div4AirportSeqID,Div4WheelsOn,Div4TotalGTime,Div4LongestGTime,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum
0,1295781,1998,2,4,2,4,1998-04-02,AS,19930,AS,N785AS,584,11884,1188401,31884,GEG,"Spokane, WA",WA,53.0,Washington,93,14747,1474702,30559,SEA,"Seattle, WA",WA,53.0,Washington,93,1330,1330.0,0.0,0.0,0.0,0.0,1300-1359,8.0,1338.0,1415.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1125375,2013,2,5,13,1,2013-05-13,EV,20366,EV,N24103,4132,11618,1161802,31703,EWR,"Newark, NJ",NJ,34.0,New Jersey,21,14524,1452401,34524,RIC,"Richmond, VA",VA,51.0,Virginia,38,1301,1255.0,-6.0,0.0,0.0,-1.0,1300-1359,9.0,1304.0,1358.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,118824,1993,3,9,25,6,1993-09-25,UA,19977,UA,NaN,2206,14108,1410801,34108,PIA,"Peoria, IL",IL,17.0,Illinois,41,13930,1393001,30977,ORD,"Chicago, IL",IL,17.0,Illinois,41,1650,1723.0,33.0,33.0,1.0,2.0,1600-1659,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,634825,1994,4,11,12,6,1994-11-12,HP,19991,HP,NaN,1207,12892,1289201,32575,LAX,"Los Angeles, CA",CA,6.0,California,91,14107,1410701,30466,PHX,"Phoenix, AZ",AZ,4.0,Arizona,81,1245,1309.0,24.0,24.0,1.0,1.0,1200-1259,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1888125,2017,3,8,17,4,2017-08-17,UA,19977,UA,N827UA,576,11003,1100303,31003,CID,"Cedar Rapids/Iowa City, IA",IA,19.0,Iowa,61,11292,1129202,30325,DEN,"Denver, CO",CO,8.0,Colorado,82,755,746.0,-9.0,0.0,0.0,-1.0,0700-0759,8.0,754.0,836.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
retard_trans = data.groupby(['Reporting_Airline', 'Year', "Month"])['CarrierDelay'].mean().reset_index()
retard_meteo = data.groupby(['Reporting_Airline', 'Year', "Month"])['WeatherDelay'].mean().reset_index()
retard_aerien = data.groupby(['Reporting_Airline', 'Year', "Month"])['LateAircraftDelay'].mean().reset_index()
retard_securite = data.groupby(['Reporting_Airline', 'Year', "Month"])['SecurityDelay'].mean().reset_index()
retard_avion = data.groupby(['Reporting_Airline', 'Year', "Month"])['ArrDelay'].mean().reset_index()

In [7]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    html.Div([
        html.H1('Dashbord Airline Us', className='card-header', style={"padding" : "25px", "textAlign": "center"}),
    ], className="bg-light"),
    
    html.Br(),
    html.Div([
        html.Div([
            html.Div(className='col-md-3'),
            html.Div([
                dcc.Dropdown(
                    id = 'annee',
                    options=[
                        {"label" : x, "value" : x} for x in data['Year'].sort_values(ascending=False).unique()
                    ],
                    placeholder='Selectionner une anneé',
                    style={"width" : '500px'}
                    ),
                ],className='col-md-6'),
            html.Div(className='col-md-3'),
        ], className='row'),
    ], className="container"),

    # html.Br(),

    html.Div([
        html.Div([
            html.Div(
                dcc.Graph(
                    id='trans', 
                figure=px.bar(retard_trans, x='Reporting_Airline', y='CarrierDelay', color="Month", title='<b>Moyenne mensuelle des retards des transporteurs par compagnie aérienne <br>déclarante pour l\'année donnée.</b>')
                ), className="col-md-6"
                ),
            html.Div(
                dcc.Graph(
                    id='meteo', 
                figure=px.bar(retard_meteo, x='Reporting_Airline', y='WeatherDelay', color="Month", title='<b>Retard météorologique moyen mensuel par compagnie aérienne déclarante <br>pour l\'année donnée.</b>')
                ), className="col-md-6"
                )
        ], className="row"),

        html.Br(),
        html.Hr(className="bg-light"),
        html.Br(),

        html.Div([
            html.Div(
                dcc.Graph(
                    id='aerien', 
                figure=px.bar(retard_aerien, x='Reporting_Airline', y='LateAircraftDelay', color="Month", title='<b>Moyenne mensuelle des retards du système aérien national par compagnie aérienne <br>déclarante pour l\'année donnée.</b>')
                ), className="col-md-6"
                ),
            html.Div(
                dcc.Graph(
                    id='securite', 
                figure=px.bar(retard_securite, x='Reporting_Airline', y='SecurityDelay', color="Month", title='<b>Moyenne mensuelle des retards de sécurité par compagnie aérienne <br>déclarante pour l\'année donnée.</b>')
                ), className="col-md-6"
                ),
        ], className="row"),

        html.Br(),
        html.Hr(className="bg-light"),
        html.Br(),

        html.Div([
            html.Div(className="col-md-3"),
            html.Div(
                dcc.Graph(
                    id='avion', 
                figure=px.bar(retard_avion, x='Reporting_Airline', y='ArrDelay', color="Month", title='<b>Moyenne mensuelle des retards des avions par compagnie aérienne <br>déclarante pour l\'année donnée.</b>')
                ), className="col-md-6"
                ),
            html.Div(className="col-md-3"),
        ], className="row"),
    ], style={'margin' : "25px"}),

    html.Br(),
], className='bg-dark')



@app.callback(
    Output('trans','figure'),
    Input('annee','value')
    )

def update_trans(annee):
    if not annee:
        return px.bar(retard_trans, x='Reporting_Airline', y='CarrierDelay', color="Month", title='<b>Moyenne mensuelle des retards du système aérien national par compagnie aérienne <br>déclarante pour l\'année donnée.</b>')
    else:
        graph = px.bar(retard_trans[retard_trans['Year'] == annee], x='Reporting_Airline', y='CarrierDelay', color="Month", title=f'<b>Moyenne mensuelle des retards du système aérien national par compagnie aérienne <br>déclarante pour l\'année donnée pour {annee}</b>')
        return graph

@app.callback(
    Output('meteo','figure'),
    Input('annee','value')
    )

def update_meteo(annee):
    if not annee:
        return px.bar(retard_meteo, x='Reporting_Airline', y='WeatherDelay', color="Month", title='<b>Retard météorologique moyen mensuel par compagnie aérienne <br>déclarante pour l\'année donnée.</b>')
    else:
        graph = px.bar(retard_meteo[retard_meteo['Year'] == annee], x='Reporting_Airline', y='WeatherDelay', color="Month", title=f'<b>Retard météorologique moyen mensuel par compagnie aérienne <br>déclarante pour l\'année donnée pour {annee}</b>')
        return graph

@app.callback(
    Output('aerien','figure'),
    Input('annee','value')
    )

def update_aerien(annee):
    if not annee:
        return px.bar(retard_aerien, x='Reporting_Airline', y='LateAircraftDelay', color="Month", title='<b>Moyenne mensuelle des retards du système aérien national par compagnie aérienne <br>déclarante pour l\'année donnée.</b>')
    else:
        graph = px.bar(retard_aerien[retard_aerien['Year'] == annee], x='Reporting_Airline', y='LateAircraftDelay', color="Month", title=f'<b>Moyenne mensuelle des retards du système aérien national par compagnie aérienne <br>déclarante pour l\'année donnée pour {annee}</b>')
        return graph

@app.callback(
    Output('securite','figure'),
    Input('annee','value')
    )

def update_securite(annee):
    if not annee:
        return px.bar(retard_securite, x='Reporting_Airline', y='SecurityDelay', color="Month", title='<b>Moyenne mensuelle des retards de sécurité par compagnie aérienne <br>déclarante pour l\'année donnée.</b>')
    else:
        graph = px.bar(retard_securite[retard_securite['Year'] == annee], x='Reporting_Airline', y='SecurityDelay', color="Month", title=f'<b>Moyenne mensuelle des retards de sécurité par compagnie aérienne <br>déclarante pour l\'année donnée pour {annee}</b>')
        return graph

@app.callback(
    Output('avion','figure'),
    Input('annee','value')
    )

def update_avion(annee):
    if not annee:
        return px.bar(retard_avion, x='Reporting_Airline', y='ArrDelay', color="Month", title='<b>Moyenne mensuelle des retards des avions par compagnie aérienne <br>déclarante pour l\'année donnée.</b>')
    else:
        graph = px.bar(retard_avion[retard_avion['Year'] == annee], x='Reporting_Airline', y='ArrDelay', color="Month", title=f'<b>Moyenne mensuelle des retards des avions par compagnie aérienne <br>déclarante pour l\'année donnée pour {annee}</b>')
        return graph

In [8]:
if __name__ == '__main__':
    app.run_server(port=8197)

Dash app running on:


<IPython.core.display.Javascript object>

## Aperçu du Dashbord
<hr>

Screenshorts

<br>

<img src="http://papabagaye.000webhostapp.com/dash.PNG">